# Exercise 2: Multi-layer perceptron in two dimensions.

In [1]:
import numpy as np
import torch
from torch.autograd import Variable

In [2]:
# fetch the data from github (might not be needed)
!test -e 02_mlp.p || wget https://github.com/WolfgangWaltenberger/vietnam2022/raw/main/02_mlp.p

In [3]:
x_data, y_data = torch.load ( "02_mlp.p" )

In [4]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear1 = torch.nn.Linear(2, 10) # 2 in and 1 out
        self.relu = torch.nn.ReLU()
        self.linear2 = torch.nn.Linear(10, 1)
        
    def forward(self, x):
        ## start with a linear layer, e.g. 2x10,
        ## apply an activation function
        ## then another linear layer with a single
        ## output node, then e.g. a sigmoid to "normalize" the output
        out1 = self.linear1(x)
        act1 = self.relu ( out1 )
        out2 = self.linear2 ( act1 )
        y_pred = torch.sigmoid( out2 )
        return y_pred

In [5]:
# Our model    
model = Model()

In [6]:
## again, define your loss function (e.g. BCELoss) and your learning method (Adam)
criterion = torch.nn.BCELoss(reduction='mean')
adam = torch.optim.Adam(model.parameters(), lr=0.003)


In [7]:
# Training loop
for epoch in range(1000):
    # Forward pass: Compute predicted y by passing x to the model
    if epoch % 100 == 0:
        print ( "epoch %d" % epoch )
    y_pred = model(x_data)
    
    # Compute and print loss
    loss = criterion(y_pred, y_data)
    
    # Zero gradients, perform a backward pass, and update the weights.
    adam.zero_grad()
    loss.backward()
    adam.step()

epoch 0
epoch 100
epoch 200
epoch 300
epoch 400
epoch 500
epoch 600
epoch 700
epoch 800
epoch 900


In [8]:
## check how correct you are (on the training data)
correct, total = 0, len(x_data)
for data, label in zip ( x_data, y_data ):
    output = model(data)
    predicted = int(output.data[0] > .5)
    label = int ( label )
    correct += (predicted == label)

print('Accuracy of the model on the %d test points: %.2f %%' % ( total, 100. * correct / total))

Accuracy of the model on the 100 test points: 99.00 %


In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import cm

fig, ax = plt.subplots()
# plot the data and separating line
nbins = 100
N2 = nbins
x_axis = np.linspace(-6, 6, nbins)
y_axis = np.linspace(-6, 6, nbins)
axes = plt.gca()
axes.set_xlim([-6,6])
axes.set_ylim([-6,6])
X, Y = np.meshgrid(x_axis, y_axis)

colors,colors_s = [], []
for i in y_data:
    if i < .5:
        colors.append ( (.0,.0,.7) )
        colors_s.append ( "blue" )
    else:
        colors.append ( (.7,.0,.0) )
        colors_s.append ( "red" )

z_axis = []

for xv in x_axis:
    tmp = []
    for yv in y_axis:
        f = model ( torch.Tensor ( [xv,yv ] ) )
        v = float ( f.data[0] )
        tmp.append ( v )
    z_axis.append ( tmp )

cp = plt.contourf ( X, Y, z_axis, levels = np.arange(0.,1.0001,.025), alpha=.7, cmap=cm.seismic )
if epoch == 0:
    cbar = fig.colorbar(cp,label="discriminator")
# colors = y_data.reshape(100)

a = plt.scatter(x_data[:,0], x_data[:,1], c= colors, linewidths=1., edgecolors='k', s=100, label = "blue data points" )
b = plt.scatter(x_data[-2:,0], x_data[-2:,1], c= colors[-2:], linewidths=1., edgecolors='k', s=100, label = "red data points" )
plt.legend() ## handles=[a,cp])
plt.title('Exercise 2: MLP in two dimensions')
plt.xlabel('$x_1$')
plt.ylabel('$x_2$')